In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed April 13 23:01:53 2022

@author: amandabreton
Allows user to pick view a species result from eventchecker.py.
Takes in the text files created by BirdNET and provides
the user with a list of the most probable bird species.
The user then tells the script which speices they would
like to investigate further and a graph is shown that
shows how the counts of that bird species changes throughout
the day.

"""

'\nCreated on Wed April 13 23:01:53 2022\n\n@author: amandabreton\nAllows user to pick view a species result from eventchecker.py.\nTakes in the text files created by BirdNET and provides\nthe user with a list of the most probable bird species.\nThe user then tells the script which speices they would\nlike to investigate further and a graph is shown that\nshows how the counts of that bird species changes throughout\nthe day.\n\n'

In [2]:
# import tool boxes 
import scipy.io.wavfile
import os
import pandas as pd
import subprocess
from subprocess import call
import argparse
import yaml
import numpy as np
import exiftool
import matplotlib.pyplot as plt

In [3]:
# %% setup your files
# NOTE: you will probably have to change this: s
csvpath = '/Users/amandabreton/Documents/GitHub/ECE-590-Identification-of-Species/audio_CSVout/'
path = '/Users/amandabreton/Documents/GitHub/ECE-590-Identification-of-Species/audio_bestbirds/'
#analyzepath = 'analyze.py'
analyzepath = '/Users/amandabreton/Documents/GitHub/BirdNET/analyze.py'
# birdnetpath = 'BirdNET/'
birdnetpath = '/Users/amandabreton/Documents/GitHub/BirdNET'
setbackpath = '/Users/amandabreton/Documents/GitHub/ECE-590-Identification-of-Species'
threshold = 0.85

In [4]:
# %% getting which files are audio
for filename in os.listdir(path):
    if filename.endswith(".wav") or filename.endswith(".WAV"):
        name = os.path.join(path, filename)
    else:
        continue

if os.path.exists(os.path.join(path, ".DS_Store")):
    os.remove(os.path.join(path, ".DS_Store"))
else:
    pass

sounds = [os.path.join(path, name) for name in os.listdir(path) if
          os.path.isfile(os.path.join(path, name))]

In [5]:
# %% putting the info into a data frame
df = pd.DataFrame(list(zip(sounds)),
                  columns=['Audio_Paths'])

# %% sending files to BirdNET analyze
os.chdir(birdnetpath)

for i in range(len(df['Audio_Paths'])):
    birdaud = str('python ') + str(analyzepath) + str(' --i ') + str(df['Audio_Paths'][i])
    subprocess.call(birdaud, shell=True)
    print('BirdNET done analyzing  audio file #' + str((i)))

print('BirdNET done analyzing ' + str(len(df['Audio_Paths']))
      + ' audio files')

FILES IN DATASET: 1 
LOADING SNAPSHOT BirdNET_Soundscape_Model.pkl ... DONE! 
BUILDING BirdNET MODEL... DONE! 
IMPORTING MODEL PARAMS... DONE! 
COMPILING THEANO TEST FUNCTION FUNCTION... DONE! 
LOADING eBIRD GRID DATA... DONE! 13800 GRID CELLS 
SID: 1 PROCESSING: plover.wav SPECIES: 987 DETECTIONS: 10 TIME: 1 
BirdNET done analyzing  audio file #0
FILES IN DATASET: 1 
LOADING SNAPSHOT BirdNET_Soundscape_Model.pkl ... DONE! 
BUILDING BirdNET MODEL... DONE! 
IMPORTING MODEL PARAMS... DONE! 
COMPILING THEANO TEST FUNCTION FUNCTION... DONE! 
LOADING eBIRD GRID DATA... DONE! 13800 GRID CELLS 
SID: 1 PROCESSING: Hummingbird.wav SPECIES: 987 DETECTIONS: 8 TIME: 2 
BirdNET done analyzing  audio file #1


usage: analyze.py [-h] [--i I] [--o O] [--filetype FILETYPE]
                  [--results RESULTS] [--lat LAT] [--lon LON] [--week WEEK]
                  [--overlap OVERLAP] [--spp SPP] [--sensitivity SENSITIVITY]
                  [--min_conf MIN_CONF]
analyze.py: error: unrecognized arguments: WingBandedAntbird.wav


BirdNET done analyzing  audio file #2


usage: analyze.py [-h] [--i I] [--o O] [--filetype FILETYPE]
                  [--results RESULTS] [--lat LAT] [--lon LON] [--week WEEK]
                  [--overlap OVERLAP] [--spp SPP] [--sensitivity SENSITIVITY]
                  [--min_conf MIN_CONF]
analyze.py: error: unrecognized arguments: Martin.wav


BirdNET done analyzing  audio file #3
FILES IN DATASET: 1 
LOADING SNAPSHOT BirdNET_Soundscape_Model.pkl ... DONE! 
BUILDING BirdNET MODEL... DONE! 
IMPORTING MODEL PARAMS... DONE! 
COMPILING THEANO TEST FUNCTION FUNCTION... DONE! 
LOADING eBIRD GRID DATA... DONE! 13800 GRID CELLS 
SID: 1 PROCESSING: CactusWren.wav SPECIES: 987 DETECTIONS: 6 TIME: 1 
BirdNET done analyzing  audio file #4
BirdNET done analyzing  audio file #5
BirdNET done analyzing 6 audio files


usage: analyze.py [-h] [--i I] [--o O] [--filetype FILETYPE]
                  [--results RESULTS] [--lat LAT] [--lon LON] [--week WEEK]
                  [--overlap OVERLAP] [--spp SPP] [--sensitivity SENSITIVITY]
                  [--min_conf MIN_CONF]
analyze.py: error: unrecognized arguments: Sparrow.wav


In [6]:
# %% Most probable birds
os.chdir(setbackpath)
files = os.listdir(path)  # Get all the files in that directory
txtfiles = []
for filename in os.listdir(path):
    if filename.endswith(".txt"):
        name = os.path.join(path, filename)
        txtfiles.append(name)
    else:
        nonimagecount = +1
        continue
# %%
knbirds = []
source = []
confidences = []
for k in range(len(txtfiles)):
    birdtxt = txtfiles[k]
    df = pd.read_csv(birdtxt, sep='\t')
    for i in range(1, len(df)):
        confid = df['Confidence'][i]
        confid = float(confid)
        if confid > threshold:
            # knbirds.append(confid)
            bird = df['Common Name'][i]
            knbirds.append(bird)
            audio = df['Begin File'][i]
            source.append(audio)
            confidences.append(confid)
        else:
            pass

df = pd.DataFrame(list(zip(knbirds, confidences, source)),
                  columns=['Species', 'Confidence', 'Audio Source'])

print('The most probable bird detections in this batch are as follows:')
print(df)

# %% graphing things
df1 = pd.DataFrame(list(zip(knbirds, source)),
                   columns=['Species', 'Audio Source'])
df1 = df1.groupby(df1.columns.tolist()).size().to_frame('Count').reset_index()

specieslist = []
[specieslist.append(x) for x in knbirds if x not in specieslist]


# %%
print('Here are the species found:')
print(specieslist)

The most probable bird detections in this batch are as follows:
                Species  Confidence    Audio Source
0           Cactus Wren    0.999124  CactusWren.wav
1           Cactus Wren    0.858004  CactusWren.wav
2  Little Ringed Plover    0.999680      plover.wav
3  Little Ringed Plover    0.999914      plover.wav
4        Kentish Plover    0.925946      plover.wav
5  Little Ringed Plover    0.999832      plover.wav
Here are the species found:
['Cactus Wren', 'Little Ringed Plover', 'Kentish Plover']
